In [1]:
!pip install git+https://github.com/modichirag/flowpm.git

  Cloning https://github.com/modichirag/flowpm.git to /tmp/pip-req-build-jn7r9yyi
  Running command git clone -q https://github.com/modichirag/flowpm.git /tmp/pip-req-build-jn7r9yyi
  Created wheel for flowpm: filename=flowpm-0.1rc2.dev257+g87ded06-cp36-none-any.whl size=33764 sha256=eb248734066501c7548d0fa0017bf211ca94995a6750909ce2ef14e740dab1ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5mj9jvp/wheels/c9/23/31/1ea68dc6cc2385226f39532d708dba63dd1c5b081acdc14705
Successfully built flowpm


In [1]:
#Setup
%tensorflow_version 1.x
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import InterpolatedUnivariateSpline as iuspline
%matplotlib inline
#
import tensorflow as tf
#
from flowpm import linear_field, lpt_init, nbody, cic_paint
import flowpm

import pandas as pd 



stages = np.linspace(0.1, 1.0, 10, endpoint=True)

data_url = 'https://raw.githubusercontent.com/modichirag/flowpm/master/flowpm/data/Planck15_a1p00.txt'

D = pd.read_table(
    data_url, 
    sep=" ", 
    index_col=False,
    skiprows=2,
    names=["a","b"]
)
D.head()

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
D = D.to_numpy()
klin = D.T[0]
plin = D.T[1]

print(klin)

[1.00000000e-05 1.00925754e-05 1.01860077e-05 ... 9.81738896e+02
 9.90827380e+02 1.00000000e+03]


In [7]:
ipklin = iuspline(klin, plin)


In [10]:
tf.reset_default_graph()

initial_conditions = flowpm.linear_field(32,          # size of the cube
                                         100,         # Physical size of the cube
                                         ipklin,      # Initial power spectrum
                                         batch_size=16)

# Sample particles
state = flowpm.lpt_init(
    initial_conditions, 
   0.1
)   



# Evolve particles down to z=0
final_state = flowpm.nbody(state, stages, 32)         

# Retrieve final density field
final_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), final_state[0])

TypeError: ignored

In [10]:
#Execute the graph!
with tf.Session() as sess:
    ic, istate, fstate, sim = sess.run([initial_conditions, state, final_state, final_field])

<tf.Tensor: shape=(16, 32, 32, 32), dtype=float32, numpy=
array([[[[5.32021150e-02, 1.24479167e-01, 2.18833119e-01, ...,
          2.60630369e-01, 3.26998770e-01, 9.25504640e-02],
         [2.36716002e-01, 2.34464332e-01, 2.97642648e-01, ...,
          3.04949313e-01, 4.36021358e-01, 4.10154104e-01],
         [2.29198307e-01, 3.10868144e-01, 2.50774294e-01, ...,
          5.11647582e-01, 5.05062282e-01, 2.28983104e-01],
         ...,
         [5.14941573e-01, 3.40685815e-01, 1.09281719e-01, ...,
          4.72049594e-01, 6.54587746e-01, 6.98288798e-01],
         [1.63349248e-02, 1.30801484e-01, 2.69778877e-01, ...,
          3.90056312e-01, 4.35828984e-01, 1.88391253e-01],
         [4.67184544e-01, 3.14416885e-01, 2.29688689e-01, ...,
          1.44781798e-01, 3.58162701e-01, 5.63905716e-01]],

        [[2.64646709e-02, 1.41825601e-01, 3.13853532e-01, ...,
          1.36037925e-02, 6.90716282e-02, 8.57420638e-02],
         [6.67055845e-01, 5.24197936e-01, 5.13621747e-01, ...,
         

In [9]:
ib = 0  #index of the Universe in the batch

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(ic[ib].sum(axis=0))
ax[0].set_title('Gaussain Initial Conditions')
ax[1].imshow(sim[ib].sum(axis=0))
ax[1].set_title('Final Evolved Universe')
plt.tight_layout()
plt.show()


RuntimeError: ignored